In [1]:
import cv2,time,pandas
from datetime import datetime

first_frame = None
status_list=[None,None]
times=[]
df=pandas.DataFrame(columns=["Start","End"])

video=cv2.VideoCapture(0)

while True:
    status = 0
    check,frame = video.read()
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur_frame = cv2.GaussianBlur(gray_frame,(21,21),0)
    
    if first_frame is None:
        first_frame = blur_frame
        continue
        
    delta_frame = cv2.absdiff(first_frame,blur_frame)
    thresh_frame = cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1]
    dilate_frame = cv2.dilate(thresh_frame, None, iterations = 5)
    
    (cnts,_) = cv2.findContours(dilate_frame.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts:
        if cv2.contourArea(contour) < 10000:
            continue
        status = 1
        (x,y,w,h) = cv2.boundingRect(contour)
        frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
        
    status_list.append(status)

    status_list=status_list[-2:]


    if status_list[-1]==1 and status_list[-2]==0:
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:
        times.append(datetime.now())  
    
    cv2.imshow("Motion Detection",frame)
    cv2.imshow("Delta Frame",delta_frame)
    cv2.imshow("Threshold Frame",thresh_frame)
    cv2.imshow("Dilated Frame",dilate_frame)
    cv2.imshow("Masked Frame",fgmask)
    
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
for i in range(0,len(times),2):
    df=df.append({"Start":times[i],"End":times[i+1]},ignore_index=True)

df.to_csv("Times.csv")

video.release()
cv2.destroyAllWindows()